In [ ]:
import os
import glob

import numpy as np # linear algebra
from scipy.stats import randint
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv), data manipulation as in SQL
import matplotlib.pyplot as plt # this is used for the plot the graph 
import seaborn as sns


import tensorflow as tf
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Activation, Flatten
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, classification_report

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier

In [ ]:
os.chdir('C:/Users/Olsen/Desktop/Masteroppgave/Code/Swell/')

# new_model = tf.keras.models.load_model('forTransferL.h5')
# #model.layers[0].trainable = False
# #x = model.layers[10].output
# new_model.summary()

# Load the pre-trained model
pretrained_model = load_model('swell_model.h5')
pretrained_model = load_model('swell_model_tree_classes.h5')

# # Display the summary of the pre-trained model
# pretrained_model.summary()


In [ ]:
# date,timestamp,hrv,dfa,sport,feeling,rpe

def replace_feeling(value):
    if value >= 6:
        return 1
    elif value == 5:
        return 2
    else:
        return 3


# def replace_feeling(value):
#     if value >= 5:
#         return 1
#     else:
#         return 0


# Function to load and preprocess data from a single file
def load_and_preprocess(file_path, session):
    df = pd.read_csv(file_path, sep=', ')

    df['Feeling'] = df['Feeling'].apply(lambda x: replace_feeling(x))

    X = df[['MEAN_RR', 'MEDIAN_RR', 'SDRR', 'RMSS', 'SDSD', 'SDRR_RMSSD', 'HR', 'pNN25', 'pNN50', 'SD1', 'SD2', 'KURT', 'SKEW', 'VLF', 'VLF_PCT', 'LF', 'LF_PCT', 'LF_NU', 'HF', 'HF_PCT', 'HF_NU', 'TP', 'LF_HF', 'HF_LF', 'sampen']]
    y = df['Feeling']
    return X, y

# Function to load and preprocess data from multiple files in a folder
def load_data_from_folder(folder_path):
    X_all, y_all = [], []

    session = 1

    for file_name in os.listdir(folder_path):
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder_path, file_name)
            X, y = load_and_preprocess(file_path, session)
            X_all.append(X)
            y_all.append(y)
            session += 1


    X_test = np.vstack(X_all)
    y_all = np.concatenate(y_all)

    return X_all, y_all

def folder_to_csv_file(folder, output_file):

    X_all = pd.DataFrame()

    for file_name in os.listdir(folder):
        if file_name.endswith(".txt"):
            file_path = os.path.join(folder, file_name)
            df = pd.read_csv(file_path, sep=', ')
            df['Feeling'] = df['Feeling'].apply(lambda x: replace_feeling(x))
            
            # pd.DataFrame.to_csv(df)
            X_all = pd.concat([X_all, df])
            # session += 1

    X_all.to_csv(output_file, index=False)
    return X_all


In [ ]:
# os.chdir('C:/Users/Olsen/Desktop/Masteroppgave/Data/fitfiler/candidate1/HRV_export/features_complete')


# train_folder = "C:/Users/Olsen/Desktop/Masteroppgave/Data/fitfiler/candidate1/HRV_export/features_complete/train/"
# train = pd.DataFrame()
# # candidate1_HRV_2023-05-18_rowing_3.0_features
# for file_name in os.listdir(train_folder):
#     if file_name.endswith(".txt"):
#         #print(file_name[-16])
#         if file_name[-16] not in ["5", "7", "3"]:
#             #print("not middle")
#             file_path = os.path.join(train_folder, file_name)
#             df = pd.read_csv(file_path, sep=', ')
#             # df['Feeling'] = df['Feeling'].apply(lambda x: replace_feeling(x))
            
#             # pd.DataFrame.to_csv(df)
#             train = pd.concat([train, df])

# train.to_csv('train.csv', index=False)

# # folder_to_csv_file(train_folder, 'train.csv')

# test_folder = "C:/Users/Olsen/Desktop/Masteroppgave/Data/fitfiler/candidate1/HRV_export/features_complete/test/"
# test = pd.DataFrame()

# for file_name in os.listdir(test_folder):
#     if file_name.endswith(".txt"):
#         if file_name[-16] not in ["5", "7", "3"]:
#             file_path = os.path.join(test_folder, file_name)
#             df = pd.read_csv(file_path, sep=', ')
#             # df['Feeling'] = df['Feeling'].apply(lambda x: replace_feeling(x))
            
#             # pd.DataFrame.to_csv(df)
#             test = pd.concat([test, df])

# test.to_csv('test.csv', index=False)

# # folder_to_csv_file(test_folder, 'test.csv')

In [ ]:
all_folder = "C:/Users/Olsen/Desktop/Masteroppgave/Data/fitfiler/candidate1/HRV_export/features_complete/dataset_v3/"
all = pd.DataFrame()



for file_name in os.listdir(all_folder):

    # all_file = str(file_name)
    # print(all_file)
    # print(all_file[-19])

    if file_name.endswith(".txt"):
        if file_name[-19] not in ["7", "3"]:
            file_path = os.path.join(all_folder, file_name)
            df = pd.read_csv(file_path, sep=', ')
            # df['Feeling'] = df['Feeling'].apply(lambda x: replace_feeling(x))
            
            # pd.DataFrame.to_csv(df)
            all = pd.concat([all, df])

all.to_csv('all_9and1.csv', index=False)

train, test = train_test_split(all, random_state=42, test_size=0.25, train_size=0.75)
# folder_to_csv_file(test_folder, 'test.csv')

In [ ]:
# # Specify the folder containing your CSV files
# train_folder = "C:/Users/Olsen/Desktop/Masteroppgave/Data/fitfiler/candidate2/csv_files"

# # Load and preprocess data from multiple files in the folder
# X_train, y_train = load_data_from_folder(train_folder)


# # Specify the folder containing your CSV files
# test_folder = "C:/Users/Olsen/Desktop/Masteroppgave/Data/fitfiler/candidate1/csv_files"

# # Load and preprocess data from multiple files in the folder
# X_test, y_test = load_data_from_folder(test_folder)

# print(X_train)
# # print(X_test)

# # Split the data into training and testing sets
# # X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, random_state=42)



# # Reshape input data for LSTM
# X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
# X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# print(X_train.shape)

In [ ]:
# os.chdir('C:/Users/Olsen/Desktop/Masteroppgave/Data/fitfiler/candidate1/csv_files')

# train_folder = "C:/Users/Olsen/Desktop/Masteroppgave/Data/fitfiler/candidate1/HRV_export/features_complete/"

# X_train, y_train = load_data_from_folder(train_folder)

print(train.shape)

In [ ]:
# X_all = pd.read_csv('v1_olt_dataset.csv')
# train, test = train_test_split(X_all)



In [ ]:
print('Test')

# Verify the changes
print(test['Feeling'].value_counts())
print(test['Feeling'].size)
print('')

print('Train')

# trainDataSet['condition'] = trainDataSet['condition'].replace({'interruption': 'stress', 'time pressure': 'stress'})

# Verify the changes
print(train['Feeling'].value_counts())
print(train['Feeling'].size)

In [ ]:
train['Feeling'] = train['Feeling'].apply(lambda x: replace_feeling(x))

y_train = train['Feeling']

y_train = pd.get_dummies(y_train)

# y_train = df_mean_train['feeling']
# labelencoder = LabelEncoder()
# y_train = labelencoder.fit_transform(df_mean_train["feeling"])
print('Train')

# trainDataSet['condition'] = trainDataSet['condition'].replace({'interruption': 'stress', 'time pressure': 'stress'})

# Verify the changes
print(train['Feeling'].value_counts())
print(train['Feeling'].size)

y_train

In [ ]:
trainSamples = train.drop(['Feeling','File'], axis = 1)

trainSamples

In [ ]:
test['Feeling'] = test['Feeling'].apply(lambda x: replace_feeling(x))

y_test = test['Feeling']
y_test = pd.get_dummies(y_test)

testlabels1 = test['Feeling']
labelencoder = LabelEncoder()
testLabels1 = labelencoder.fit_transform(test['Feeling'])

print('Test')

# Verify the changes
print(test['Feeling'].value_counts())
print(test['Feeling'].size)
print('')

# y_train = df_mean_train['feeling']
# labelencoder = LabelEncoder()
# y_train = labelencoder.fit_transform(df_mean_train["feeling"])

y_test

In [ ]:
testSamples = test.drop(['Feeling','File'], axis = 1)

testSamples

In [ ]:
# Display the summary of the pre-trained model
pretrained_model.summary()



In [ ]:
# # input_shape = (8,)

# # Create a new model for transfer learning
# transfer_model = Sequential()

# # transfer_model.add(Dense(8, activation='sigmoid', input_shape=input_shape, name='dense_input'))

# for layer in pretrained_model.layers[:-1]:
#     transfer_model.add(layer)


# transfer_model.add(Dense(3, activation='softmax', name='output_layer'))  # Modify activation based on your problem


# for layer in pretrained_model.layers[:-2]:
#     layer.trainable = True


# transfer_model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
# transfer_model.compile(optimizer='Adam', loss='mse', metrics=['accuracy'])

# transfer_model.summary()

# # transfer_model.fit(trainSamples, y_train, epochs=40, batch_size=32, validation_split=0.1)
# transfer = transfer_model.fit(trainSamples, y_train, validation_split=0.1 ,batch_size=100, epochs=100, verbose=1, shuffle=True)
# # early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# pretrained = pretrained_model.fit(trainSamples, y_train, validation_split=0.1,  batch_size=100, epochs=100, verbose=1, shuffle=True)
# # transfer_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1, callbacks=[early_stopping])

# # transfer_model.save('transfer_model.h5')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.applications import YourPretrainedModel  # Replace with the actual pre-trained model you're using

# Assuming input_shape is defined based on your specific problem
input_shape = (your_input_shape_here)

# Load the pre-trained model
pretrained_model = YourPretrainedModel(weights='imagenet', include_top=False, input_shape=input_shape)

# Create a new model using the pre-trained weights
pretrained_only_model = Sequential()
pretrained_only_model.add(pretrained_model)
pretrained_only_model.add(Dense(3, activation='softmax', name='output_layer'))  # Modify activation based on your problem

# Compile the model
pretrained_only_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])  # Modify loss based on your problem

# Display the model summary
pretrained_only_model.summary()

# Train the model on your data
history = pretrained_only_model.fit(trainSamples, y_train, validation_split=0.1, batch_size=100, epochs=100, verbose=1, shuffle=True)


In [ ]:
# summarize history for accuracy
plt.plot(pretrained.history['accuracy'])
plt.plot(pretrained.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for accuracy
plt.plot(transfer.history['accuracy'])
plt.plot(transfer.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# # Create a new model for transfer learning
# transfer_model = Sequential()

# # Add a Flatten layer to reshape the input
# # transfer_model.add(Flatten(input_shape=(1, 9)))


# # Add layers to the transfer model by iterating through the pre-trained model layers
# for layer in pretrained_model.layers[:-1]:
#     # Add a layer with the same configuration, excluding the input shape
#     transfer_model.add(layer.__class__.from_config(layer.get_config()))

# # Add a new output layer with the correct number of units for your specific problem
# transfer_model.add(Dense(1, activation='sigmoid'))  # Modify activation based on your problem


# # Compile the transfer model
# transfer_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Display the summary of the transfer model
# transfer_model.summary()

# # Assuming your training data is stored in X_train and labels in y_train
# # Make sure to adjust the parameters in fit() according to your requirements
# transfer_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2)



In [ ]:
# test_loss, test_acc = pretrained_model.evaluate(testSamples,  y_test, verbose=2)
# print('\nTest loss: ', test_loss)
# print('\nTest accuracy: ', test_acc)


In [ ]:
# test_loss, test_acc = transfer_model.evaluate(testSamples,  y_test, verbose=2)
# print('\nTest loss: ', test_loss)
# print('\nTest accuracy: ', test_acc)


## Predictions

### Squential

In [ ]:


# Make predictions
# probability_model = tf.keras.Sequential([transfer_model, tf.keras.layers.Softmax()])
# y_predict = probability_model.predict(testSamples)
# # predictions = model.predict(X_test)


y_predict = pretrained_model.predict(testSamples)
print(y_predict)
y_pred = np.argmax(y_predict, axis = -1)


t =pd.DataFrame(y_pred)
t.value_counts()
# df_mean_test.feeling.value_counts()

from sklearn.metrics import confusion_matrix
confusion = confusion_matrix(y_pred,testLabels1)
print('Confusion Matrix\n')
print(confusion)

In [ ]:


accuracy = accuracy_score(testLabels1, y_pred)
print("Accuracy:", accuracy)

report = classification_report(testLabels1, y_pred, zero_division=np.nan)
print("Classification Report:")
print(report)

# confusion = confusion_matrix(testLabels1, y_pred) # add normalize='pred' if want %
# print('Confusion Matrix\n')
# print(confusion)

ax = sns.heatmap(confusion, annot=True, cmap='Blues')

ax.set_title(' Confusion Matrix with labels\n\n')
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ')

Transfer Model


In [ ]:


# Make predictions
# probability_model = tf.keras.Sequential([transfer_model, tf.keras.layers.Softmax()])
# y_predict = probability_model.predict(testSamples)
# # predictions = model.predict(X_test)


# y_predict = transfer_model.predict(testSamples)
# print(y_predict)
# y_pred = np.argmax(y_predict, axis = -1)


# t =pd.DataFrame(y_pred)
# t.value_counts()
# # df_mean_test.feeling.value_counts()

# from sklearn.metrics import confusion_matrix
# confusion = confusion_matrix(y_pred,testLabels1)
# print('Confusion Matrix\n')
# print(confusion)

In [ ]:
# accuracy = accuracy_score(testLabels1, y_pred)
# print("Accuracy:", accuracy)

# report = classification_report(testLabels1, y_pred, zero_division=np.nan)
# print("Classification Report:")
# print(report)

# confusion = confusion_matrix(testLabels1, y_pred) # add normalize='pred' if want %
# print('Confusion Matrix\n')
# print(confusion)

# ax = sns.heatmap(confusion, annot=True, cmap='Blues')

# ax.set_title(' Confusion Matrix with labels\n\n')
# ax.set_xlabel('\nPredicted Values')
# ax.set_ylabel('Actual Values ')

### Random Forest Classifier

In [ ]:
rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=15, min_samples_leaf=1, random_state=42)
rf_classifier.fit(trainSamples, y_train)
y_pred = rf_classifier.predict(testSamples)
y_roundedValue = np.argmax(y_pred, axis = -1)
# print(y_pred)
accuracy = accuracy_score(testLabels1, y_roundedValue)
print("Accuracy:", accuracy)

report = classification_report(testLabels1, y_roundedValue)
print("Classification Report:")
print(report)

result = confusion_matrix(testLabels1, y_roundedValue)

ax = sns.heatmap(result, annot=True, cmap='Blues')

ax.set_title(' Confusion Matrix with labels\n\n')
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ')



# result = confusion_matrix(y_test, y_pred , normalize='pred')
# print(classification_report(y_test, y_pred))

# disp = ConfusionMatrixDisplay(confusion_matrix=result)

# disp.plot(cmap=plt.cm.Blues)
# plt.show()

In [ ]:

# # Evaluate the model
# loss = transfer_model.evaluate(testSamples, y_test)
# print(f"Test Loss: {loss}")

# test_loss, test_acc = transfer_model.evaluate(testSamples,  y_test, verbose=2)
# print('\nTest loss: ', test_loss)
# print('\nTest accuracy: ', test_acc)


### Decision Tree Classifier

In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(trainSamples, y_train)
y_pred = rf_classifier.predict(trainSamples)
y_roundedValue = np.argmax(y_pred, axis = -1)

accuracy = accuracy_score(testLabels1, y_roundedValue)
print("Accuracy:", accuracy)

report = classification_report(testLabels1, y_roundedValue)
print("Classification Report:")
print(report)


result = confusion_matrix(testLabels1, y_roundedValue)
print(classification_report(y_test, y_predict))

disp = ConfusionMatrixDisplay(confusion_matrix=result)

disp.plot(cmap=plt.cm.Blues)
plt.show()

### K Nearest Neighbors Classifier

In [ ]:
knn_classifier = KNeighborsClassifier(n_neighbors=1000)
knn_classifier.fit(trainSamples, y_train)
y_pred = rf_classifier.predict(trainSamples)
y_roundedValue = np.argmax(y_pred, axis = -1)

accuracy = accuracy_score(testLabels1, y_roundedValue)
print("Accuracy:", accuracy)

report = classification_report(testLabels1, y_roundedValue)
print("Classification Report:")
print(report)


result = confusion_matrix(testLabels1, y_roundedValue)
print(classification_report(testLabels1, y_roundedValue))

disp = ConfusionMatrixDisplay(confusion_matrix=result)

disp.plot(cmap=plt.cm.Blues)
plt.show()

### Gradient Boosting Classifier

In [ ]:
gb_classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_classifier.fit(trainSamples, y_train)
y_pred = rf_classifier.predict(trainSamples)
y_roundedValue = np.argmax(y_pred, axis = -1)

accuracy = accuracy_score(testLabels1, y_roundedValue)
print("Accuracy:", accuracy)

report = classification_report(testLabels1, y_roundedValue)
print("Classification Report:")
print(report)


result = confusion_matrix(testLabels1, y_roundedValue)


disp = ConfusionMatrixDisplay(confusion_matrix=result)

disp.plot(cmap=plt.cm.Blues)
plt.show()

In [ ]:
# model.save('forTransferL.h5')